In [2]:
USERNAME='
PASSWORD='

In [6]:
from selenium import webdriver
import time

# profile = webdriver.ChromeProfile() # 新增firefox的設定
# profile.set_preference("dom.webnotifications.enabled", False) # 將頁面通知關掉
driver = webdriver.Chrome()
driver.get("http://www.facebook.com")
time.sleep(3)
driver.find_element_by_id("email").send_keys(USERNAME) # 將USERNAME改為你的臉書帳號
driver.find_element_by_id("pass").send_keys(PASSWORD) # 將PASSWORD改為你的臉書密碼
driver.find_element_by_id("loginbutton").click()
time.sleep(3)
driver.get('https://www.facebook.com/groups/1517250658578705/')

In [9]:
for i in range(12): # 捲動12次
    driver.execute_script("window.scrollTo(0, {})".format(4000 * (i + 1))) #每次捲動4000的單位
    time.sleep(2) # 等待2秒鐘讓頁面讀取

In [10]:
def ClickForMore():
    hrefBtns = driver.find_elements_by_tag_name('a')    
    for btn in hrefBtns:
        try:
            s = btn.get_attribute('data-testid')
        except:
            continue
        if s == 'UFI2CommentsPagerRenderer/pager_depth_1' or s == 'UFI2CommentsPagerRenderer/pager_depth_0':
            try:
                btn.click()
                time.sleep(1)
            except:
                continue
ClickForMore()
ClickForMore()

In [13]:
from bs4 import BeautifulSoup
import datetime
import re

htmltext = driver.page_source # 將網頁原始碼拿出來


def parse_htmltext(htmltext, start_date, end_date):
    '''
    解析臉書貼文與回覆的原始碼。
    htmltext為原始碼，str
    star_date為起始日期，datetime.datetime
    end_date為結束日期，datetime.datetime
    '''   
    post_persons = []
    comment_persons = []
    good_urllist = [] # 回復表情符號超連結
    ustart_date = start_date.timestamp()
    uend_date = end_date.timestamp()
    soup = BeautifulSoup(htmltext, 'html.parser')
    body = soup.find('body')
    posts = body.select('div[id="pagelet_group_mall"]')[0].select('div[aria-label="動態消息"]')[0]
    feed_articles = posts.select('div[role="feed"]')[0].select('div[role="article"]')
    other_articles = posts.select('div[role="article"]')
    articles = feed_articles + other_articles # 所有貼文或留言
    
    for article in articles:
        if article.has_attr('id'):
            try:
                post_person = re.findall('title="(.{2,20})"><div class=', str(article))[0]
            except:
                continue
            post_time = int(re.findall('data-utime="(.*?)"', str(article))[0])        
            if post_time >= ustart_date and post_time <= uend_date:                
                post_persons.append(post_person)
            try:
                good_urllist.append(re.findall('"(/ufi/reaction/profile/browser/\?.*?)"', str(article))[0])
            except:
                pass
    
        elif article.has_attr('data-testid'):            
            comment_person = re.findall('directed_target_id.*?href=".*?">(.*?)</a>', str(article))[0]  
            comment_time = int(re.findall('data-utime="(.*?)"', str(article))[0])
            if comment_time >= ustart_date and post_time <= uend_date:                    
                comment_persons.append(comment_person)                    
                try:
                    good_urllist.append(re.findall('"(/ufi/reaction/profile/browser/\?.*?)"', str(article))[0])
                except:
                    pass
    
    return post_persons, comment_persons, good_urllist

post_persons, comment_persons, good_urllist = parse_htmltext(htmltext, datetime.datetime(2020, 3, 25), datetime.datetime(2020, 3, 30))

In [14]:
post_persons

['Wen Hua Yen',
 'Wen Hua Yen',
 'Yung Chnu',
 'Mei Ling',
 'Daniel Wse',
 'Richard Wang',
 'Yi L',
 'Ray Hsiao',
 'Feng Liu',
 '藍小雨',
 'Gary Lai',
 'Damon Wu',
 '若凡',
 '陳佩蓉',
 '呂郁青',
 '洪柏璋',
 '錢宗右',
 'Ray Hsiao',
 'Yung Chnu',
 '陳佐',
 '鄭瑪答',
 'Jennifer Pai',
 'Dan Chen',
 'Kevin Chen',
 'Moni Ho',
 '郭軒旻',
 'Pension Sherlock',
 'Sharon Yang',
 'HO Will',
 'Ray Hsiao',
 'Chiang Fred',
 'Lily Wang',
 'Yi Ting Chen',
 '劉靖宇',
 '劉靖宇',
 'Ben Chen',
 'Ben Chen',
 'Chang Ching Chi',
 'Gary Lai',
 '陳佐',
 'HO Will',
 '鄭瑪答',
 'Peter Quill',
 '陳瑞典',
 'Lonsen Shen',
 'Tony Hsieh',
 '劉靖宇',
 '吳宇崴',
 '張時杰']

In [22]:
def parse_good_urllist(urllist):
    
    output = []

#     profile = webdriver.FirefoxProfile()
#     profile.set_preference("dom.webnotifications.enabled", False)
#     profile.update_preferences()
    driver = webdriver.Chrome()
    driver.get("http://www.facebook.com")
    driver.find_element_by_id("email").send_keys(USERNAME) # 將USERNAME改為你的臉書帳號
    driver.find_element_by_id("pass").send_keys(PASSWORD) # 將PASSWORD改為你的臉書帳號
    driver.find_element_by_id("loginbutton").click()
    time.sleep(3)

    for url in urllist:
        driver.get('http://www.facebook.com/' + url)
        htmltext = driver.page_source
        soup = BeautifulSoup(htmltext, 'html.parser')
        for raw_text in soup.select('li[class="_5i_q"]'):
            output += re.findall(re.compile('aria-label="(.*?)" class="_s'),str(raw_text))            

    driver.close()
    return output

emoji_replies = parse_good_urllist(good_urllist)

In [34]:
import pandas as pd

def tidy_up_data(post_persons, comment_persons, emoji_persons):
    
    all_persons = list(set(post_persons+comment_persons+emoji_persons))
    post_times = []
    comment_times = []
    emoji_times = []
    
    for p in all_persons:
        post_times.append(post_persons.count(p))
        comment_times.append(comment_persons.count(p))
        emoji_times.append(emoji_persons.count(p))
    
    return pd.DataFrame({'成員ID':all_persons, '貼文次數':post_times, '回文次數':comment_times, '回覆表情符號次數':emoji_times})
        
df = tidy_up_data(post_persons, comment_persons, emoji_replies)
# df.to_excel('member_activity.xlsx', index=False)

In [36]:
df.to_excel('member_activity.xlsx', index=False)